In [16]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Evan Barnes (ebarnes@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [17]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
# from arcgis.gis import GIS
from time import time
arcpy.CheckOutExtension('Spatial')
# import rasterio

'CheckedOut'

In [18]:
arcpy.env.overwriteOutput = True

In [19]:
path = "P:\Temp\jMatney\Indiana_ML_2"

In [5]:
st = time()
# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

#set source for shapefile with all 11 subwatersheds in the county

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(path, "50sws_10groups\\wbdhuc12_15Jason_final5_2.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Indiana_50c_streams_edited.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "Indiana_3dep\Indiana_dem2")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_slope2")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")

#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")

#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

0.15 minutes to process.


In [6]:
subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []

In [7]:
print(all_subwatersheds)

P:\Temp\jMatney\Indiana_ML_2\50sws_10groups\wbdhuc12_15Jason_final5_2.shp


In [9]:
#add AREA and PERIMETER fields for the whole subwatersheds file
#calculate area of subwatersheds
arcpy.AddField_management(all_subwatersheds, "AREA", "DOUBLE")

arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "AREA AREA_GEODESIC", '', 
                                                    "SQUARE_KILOMETERS",
                                                    None)
#calculate perimeter of subwatershed
arcpy.AddField_management(all_subwatersheds, "PERIMETER", "DOUBLE")
perimeter = arcpy.CalculateGeometryAttributes_management(all_subwatersheds, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                         "KILOMETERS")

print("perimeter calculated")

perimeter calculated


In [10]:
st = time()
# perform all need intersections here:

#put all intersections in this projection so area and length will be in meters
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(102008)

# nfhl_sfha_intersect = "nfhl_sfha.shp"
# nfhl_sfha_intersect = arcpy.Intersect_analysis([nfhl_sfha, all_subwatersheds], nfhl_sfha_intersect)

water_bodies_intersect = "water_bodies.shp"
water_bodies_intersect = arcpy.Intersect_analysis([water_bodies, all_subwatersheds], water_bodies_intersect)

# dams_intersect = "dams.shp"
# arcpy.Intersect_analysis([dams, all_subwatersheds], dams_intersect)

# bridges_intersect = "bridges.shp"
# arcpy.Intersect_analysis([bridges, all_subwatersheds], bridges_intersect)

# streets_intersect = "streets.shp"
# arcpy.Intersect_analysis([streets, all_subwatersheds], streets_intersect)

# railroads_intersect = "railroads.shp"
# arcpy.Intersect_analysis([railroads, all_subwatersheds], railroads_intersect)

population_intersect = "population.shp"
arcpy.Intersect_analysis([population, all_subwatersheds], population_intersect)

median_income_intersect= "median_income.shp"
arcpy.Intersect_analysis([median_income, all_subwatersheds], median_income_intersect)

building_footprints_intersect = "building_footprints.shp"
arcpy.Intersect_analysis([building_footprints, all_subwatersheds], building_footprints_intersect)

streams_intersect = "streams.shp"
arcpy.Intersect_analysis([streams, all_subwatersheds], streams_intersect)

# lu_subwatershed = "lu_subwatershed.shp"
# arcpy.Intersect_analysis([lu_usa, all_subwatersheds], lu_subwatershed)

print(round(((time()-st)/60), 2) , 'minutes to process.')

20.36 minutes to process.


In [11]:
def get_length_area(layer, length_area):
    #length_Area = 'LENGTH' or 'AREA'
    search_field = "SHAPE@{}".format(length_area)
    values = []
    with arcpy.da.SearchCursor(layer, [search_field]) as sCursor: 
        for s in sCursor:
            # convert meters to kilometers
            if length_area == 'AREA':
                value = s[0]/ (1000 * 1000)
            else: # assumed length
                value = s[0] / 1000
                    
            values.append(value)
    return values

In [12]:
st = time()
# make all layers for selections
#subwatershed_selection = "subwatershed_lyr"
#arcpy.MakeFeatureLayer_management (all_subwatersheds, subwatershed_selection)

# nfhl_sfha_selection = "nfhl_sfha_lyr"
# arcpy.MakeFeatureLayer_management(nfhl_sfha_intersect, nfhl_sfha_selection)

water_bodies_selection = "water_bodies_lyr"
arcpy.MakeFeatureLayer_management(water_bodies_intersect, water_bodies_selection)

# dams_selection = "dams_lyr"
# arcpy.MakeFeatureLayer_management(dams_intersect, dams_selection)

# bridges_selection = "bridges_lyr"
# arcpy.MakeFeatureLayer_management(bridges_intersect, bridges_selection)

# streets_selection = "streets_lyr"
# arcpy.MakeFeatureLayer_management(streets_intersect, streets_selection)

# railroads_selection = "railroads_lyr"
# arcpy.MakeFeatureLayer_management(railroads_intersect, railroads_selection)

population_selection = "population_lyr"
arcpy.MakeFeatureLayer_management(population_intersect, population_selection)

median_income_selection = "median_income_lyr"
arcpy.MakeFeatureLayer_management(median_income_intersect, median_income_selection)

building_footprints_selection= "building_footprints_lyr"
arcpy.MakeFeatureLayer_management(building_footprints_intersect, building_footprints_selection)

streams_selection = "streams_lyr"
arcpy.MakeFeatureLayer_management(streams_intersect, streams_selection)

# lu_selection = "lu_lyr"
# arcpy.MakeFeatureLayer_management(lu_subwatershed, lu_selection)

print(round(((time()-st)/60), 2) , 'minutes to process.')

11.03 minutes to process.


In [13]:
#arcpy.GetCount_management(streams_selection)

In [14]:
st = time()


#search cursor through each row of county subwatersheds file
with arcpy.da.SearchCursor(all_subwatersheds, ['HUC12','AREA', 'PERIMETER']) as cursor:

    for row in cursor:
        subwatershed_number = row[0]
        
        subwatershed_list.append(subwatershed_number)

        print("--------------------", subwatershed_number, " is the subwatershed to be worked on ------------------------")
        # Make a layer from the feature class
        arcpy.MakeFeatureLayer_management(all_subwatersheds,"subwatersheds_lyr")
        
        #select subwatershed from shapefile with all of them
        s = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION", 
                                                "HUC12 = '{}'".format(subwatershed_number))
        arcpy.CopyFeatures_management(s, "s_lyr.shp")
        subwatershed_selection1 = os.path.join(path, "working_dsn\s_lyr.shp")
        #arcpy.MakeFeatureLayer_management(s, subwatershed_selection1)
        
        # use mask in lieu of clipping rasters
        arcpy.env.cellSize = dem_slope
        arcpy.env.snapRaster = dem_slope
        #arcpy.env.outputCoordinateSystem = 
        arcpy.env.mask = subwatershed_selection1

        area = row[1]
        area_list.append(area)
        print(area, " square kilometers")

        perimeter = row[2]
        perimeter_list.append(perimeter)
        print(perimeter, " kilometers")


        #calculate area covered by lakes/reserviors 
        arcpy.management.SelectLayerByAttribute(water_bodies_selection, "NEW_SELECTION", 
                                                            "HUC12 = '{}'".format(subwatershed_number))
        water_bodies_area_sum = sum(get_length_area(water_bodies_selection, "AREA"))
        water_bodies_list.append(water_bodies_area_sum)
        print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")


        #all streams in subwatershed
        arcpy.management.SelectLayerByAttribute(streams_selection, "NEW_SELECTION", 
                                                            "HUC12 = '{}'".format(subwatershed_number))
        print("streams clipped")
        print("length calculated")
        #get the sum of all of the areas
        stream_length_list = get_length_area(streams_selection, "LENGTH")  
        stream_length_sum = sum(stream_length_list)
        print(stream_length_sum, " = length of all streams in subwatershed")
        print(area, " = subwatershed area")
        drainage_density = stream_length_sum / area
        drainage_density_list.append(drainage_density)
        print(drainage_density, "is the drainage density (streams/km)")
        #find watershed length
        #watershed length = distance from outlet to watershed boundary along the main channel
        #we are assuming that the longest stream above is the main channel
        watershed_length = max(stream_length_list)
        watershed_length_list.append(watershed_length)
        print(watershed_length, " = watershed length (longest stream in subwatershed)")
        #calculate shape factor 
        #watershed length squared divided by watershed area
        shape_factor = (watershed_length**2) / area
        shape_factor_list.append(shape_factor)
        print(shape_factor, " is the shape factor.")

        
        #calculate relief ratio
        #relief divided by watershed length
        #length is is kilometers, convert to meters
        watershed_length_meters = watershed_length * 1000
        relief_ratio = relief / watershed_length_meters
        relief_ratio_list.append(relief_ratio)
        print(relief_ratio, " = relief ratio")


        #calculate ruggedness number
        #product of relief and drainage density
        #relief is in meters, convert first to km
        relief_km = relief / 1000
        print(relief_km, " is the relief in km")
        ruggedness = relief_km * drainage_density
        ruggedness_list.append(ruggedness)
        print(ruggedness, " is the ruggedness number")
        
        
                #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
        #data is chosen to be on census tract level
        arcpy.management.SelectLayerByAttribute(population_selection, "NEW_SELECTION", 
                                                            "HUC12 = '{}'".format(subwatershed_number))
        #get total population
        #field = B01001_001E = Total Population (alias)
        #when clipped, field name changes to B01001_001

        #get the sum of all of the populations in each tract
        total_pop_list = []  
        with arcpy.da.SearchCursor(population_selection, ['B01001_001']) as popCursor:  
            for pop in popCursor:  
                total_pop_value = pop[0] 
                total_pop_list.append(total_pop_value)  
        total_pop_sum = sum(total_pop_list)
        population_list.append(total_pop_sum)
        print(total_pop_sum, " = total population in the subwatershed") 

        #get the average of percentages of dependent age groups in each tract
        #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
        #when clipped field changes to B01001_61
        dependent_pop_list = []  
        with arcpy.da.SearchCursor(population_selection, ["B01001__61"]) as dpopCursor: 
            for dpop in dpopCursor:  
                dependent_pop_value = dpop[0] 
                dependent_pop_list.append(dependent_pop_value)  
        dependent_pop_avg_pct = mean(dependent_pop_list)
        dependent_population_list.append(dependent_pop_avg_pct)

        print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 

        #find population density
        population_density = total_pop_sum / area
        population_density_list.append(population_density)
        print(population_density, " = population density of subwatershed (people/square km)")
        

        #get housing density
        #select buildings that are marked residential
        arcpy.SelectLayerByAttribute_management(building_footprints_selection, "NEW_SELECTION",
                                                                             "HUC12 = '{}' And RES_NONRES = 'Res'".format(subwatershed_number))
        #get count of how many buildings there are 
        buildings_count = arcpy.GetCount_management(building_footprints_selection)
        buildings_count_number = buildings_count.getOutput(0)
        print(buildings_count_number, " = number of residential building footprints in the subwatershed")
        #divide number of buildings by subwatershed area
        housing_density = int(buildings_count_number) / area
        housing_density_list.append(housing_density)
        print(housing_density, " = housing density (buildings per square km)")

    

    #########################################################################################




print(round(((time()-st)/60), 2) , 'minutes to process.')    


-------------------- 051201100605  is the subwatershed to be worked on ------------------------
65.5745457799  square kilometers
51.5520976498  kilometers
3.43634343147278  = avg slope (%)
0.3100648045502216  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
267.019165039
143.164840698
The relief of the subwatershed is:  123.85432434099997  meters.
0  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
0.033727433669857004  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
6  = number of bridges in the subwatershed
71.2774387253646  = sum of all streets in the subwatershed (km)
0  = sum of all railroads in the subwatershed (km)
9157  = total population in the subwatershed
39.2  = total percent of dependent population in the subwatershed
143.31484674210844  = population density of subwatershed (people/square km)
61160.5  = average median income in the

C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\core\fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\core\_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\core\_methods.py:140: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\core\_methods.py:110: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\numpy\core\_methods.py:132: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


nan
nan
orb50yr24ha_am
5873.6000976563  = avg rainfall in inches *1000
orb2yr24ha_am
2833.4946289063  = avg rainfall in inches *1000
orb100yr24ha_am
6576.4106445313  = avg rainfall in inches *1000
orb25yr24ha_am
5198.86328125  = avg rainfall in inches *1000
-------------------- 051201011404  is the subwatershed to be worked on ------------------------
41.8061957163  square kilometers
31.8306437973  kilometers
2.57881140708923  = avg slope (%)
0.5185130224434525  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
246.427993795
152.32940696
The relief of the subwatershed is:  94.09858683499999  meters.
0.08833485184618661  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
0.09285627129804307  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
5  = number of bridges in the subwatershed
52.68174128233048  = sum of all streets in the subwatershed (km)
0  

In [15]:
st = time()
outputs = {'subwatershed': subwatershed_list,
          'area': area_list,
          'perimeter': perimeter_list,
          'circulatory_ratio': circulatory_ratio_list,
          'relief': relief_list,
          'avg_slope': avg_slope_list,
           'watershed_length': watershed_length_list,
           'elongation_ratio': elongation_ratio_list,
           'drainage_density': drainage_density_list,
           'shape_factor': shape_factor_list,
           'relief_ratio': relief_ratio_list,
           'ruggedness': ruggedness_list,
           'aae_area': aae_list,
           'buildings_aae_count': buildings_aae_list,
           'x_area': x_list,
           'buildings_x_count': buildings_x_list,
           'water_bodies_area': water_bodies_list,
           'dams_count': dams_list,
           'bridges_count': bridges_list,
           'streets_km': streets_list,
           'railroads_km': railroads_list,
           'population': population_list,
           'population_density': population_density_list,
           'avg_median_income': avg_median_income_list,
           'housing_density': housing_density_list,
           'population_change': population_change_list,
           'dependent_population_pct': dependent_population_list,
           'dist_to_stream_avg (m)': dist_to_stream_avg_list,
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list,
           'lu_21_area' : lu_21_list,
           'lu_22_area' : lu_22_list,
           'lu_23_area' : lu_23_list,
           'lu_24_area': lu_24_list,
           'lu_41_area': lu_41_list,
           'lu_82_area': lu_82_list,
           'avg_impervious_percent': impervious_percent_list,
#            'orb100yr06h': orb100yr06h_list,
#            'orb100yr12h': orb100yr12h_list,
#            'orb100yr24h': orb100yr24h_list,
#            'orb25yr06h': orb25yr06h_list,
#            'orb25yr12h': orb25yr12h_list,
#            'orb25yr24h':orb25yr24h_list,
#            'orb2yr06h': orb2yr06h_list,
#            'orb2yr12h': orb2yr12h_list,
#            'orb2yr24h': orb2yr24h_list,
#            'orb50yr06h': orb50yr06h_list,
#            'orb50yr12h': orb50yr12h_list,
#            'orb50yr24h':orb50yr24h_list,
#            'orb100yr06ha_am': orb100yr06ha_am_list,
#            'orb100yr12ha_am': orb100yr12ha_am_list,
           'orb100yr24ha_am': orb100yr24ha_am_list,
#            'orb25yr06ha_am': orb25yr06ha_am_list,
#            'orb25yr12ha_am': orb25yr12ha_am_list,
           'orb25yr24ha_am': orb25yr24ha_am_list,
#            'orb2yr06ha_am': orb2yr06ha_am_list,
#            'orb2yr12ha_am': orb2yr12ha_am_list,
           'orb2yr24ha_am': orb2yr24ha_am_list,
#            'orb50yr06ha_am': orb50yr06ha_am_list,
#            'orb50yr12ha_am': orb50yr12ha_am_list,
           'orb50yr24ha_am': orb50yr24ha_am_list
          }

# 
# 
# 
#  
#         
# 
            
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                             'area',
                                             'perimeter',
                                             'circulatory_ratio',
                                             'relief',
                                             'avg_slope',
                                              'watershed_length',
                                              'elongation_ratio',
                                              'drainage_density',
                                              'shape_factor',
                                              'relief_ratio',
                                              'ruggedness',
                                              'aae_area',
                                              'buildings_aae_count',
                                              'x_area',
                                              'buildings_x_count',
                                              'water_bodies_area',
                                              'dams_count',
                                              'bridges_count',
                                              'streets_km',
                                              'railroads_km',
                                              'population',
                                              'population_density',
                                              'avg_median_income',
                                              'housing_density',
                                              'population_change',
                                              'dependent_population_pct',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)',
                                              'lu_21_area',
                                              'lu_22_area',
                                              'lu_23_area',
                                              'lu_24_area',
                                              'lu_41_area',
                                              'lu_82_area',
                                              'avg_impervious_percent',
                                             'orb100yr06h',
                                              'orb100yr12h',
                                              'orb100yr24h',
                                              'orb25yr06h',
                                              'orb25yr12h',
                                              'orb25yr24h',
                                              'orb2yr06h',
                                              'orb2yr12h',
                                              'orb2yr24h',
                                              'orb50yr06h',
                                              'orb50yr12h',
                                              'orb50yr24h',
                                              'orb100yr06ha_am',
                                              'orb100yr12ha_am',
                                              'orb100yr24ha_am',
                                              'orb25yr06ha_am',
                                              'orb25yr12ha_am',
                                              'orb25yr24ha_am',
                                              'orb2yr06ha_am',
                                              'orb2yr12ha_am',
                                              'orb2yr24ha_am',
                                              'orb50yr06ha_am',
                                              'orb50yr12ha_am',
                                              'orb50yr24ha_am'
                                             ])    
    

#     
#  
# 
# 
# 
#




print(outputs_df)

outputs_df.to_excel(os.path.join(path,"working_dsn\\wbdhuc12_15Jason_final5_2.xlsx"))
print(round(((time()-st)/60), 2) , 'minutes to process.')

   subwatershed       area  perimeter  circulatory_ratio      relief  \
0  051201100605  65.574546  51.552098           0.310065  123.854324   
1  051201011404  41.806196  31.830644           0.518513   94.098587   

          avg_slope  watershed_length  elongation_ratio  drainage_density  \
0  3.43634343147278         24.601327          0.366630          0.385032   
1  2.57881140708923         10.843056          0.005984       6006.103938   

   shape_factor  ...  orb100yr24ha_am  orb25yr06ha_am  orb25yr12ha_am  \
0      9.472291  ...  6576.4106445313             NaN             NaN   
1  35561.288520  ...  6083.4677734375             NaN             NaN   

    orb25yr24ha_am  orb2yr06ha_am orb2yr12ha_am    orb2yr24ha_am  \
0    5198.86328125            NaN           NaN  2833.4946289063   
1  4787.5966796875            NaN           NaN  2570.4677734375   

  orb50yr06ha_am orb50yr12ha_am   orb50yr24ha_am  
0            NaN            NaN  5873.6000976563  
1            NaN        

In [ ]:
subwatershed_list[0:3]